<a href="https://colab.research.google.com/github/lmcanavals/pcd/blob/master/spin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!gcc

gcc: fatal error: no input files
compilation terminated.


In [ ]:
!sudo apt install spin

In [4]:
!spin

Spin Version 6.4.6 -- 2 December 2016
spin: error, no filename specified


In [102]:
%%file prodcons1.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

#define MAX 5

int buffer[MAX]
byte pos = 0

byte notEmpty = 0

inline append(d) {
    assert pos < MAX
    atomic {
        buffer[pos] = d
        pos++
    }
}

inline take(d) {
    atomic {
        pos--
        d = buffer[pos]
    }
}

inline produce(d) {
    d++
}

active proctype Producer() {
    int d
    do
    ::
      //produce(d)
      append(d)
      signal(notEmpty)
    od
}

active proctype Consumer() {
    int d
    do
    ::
      wait(notEmpty)
      take(d)
    od
}

Overwriting prodcons1.pml


In [103]:
%%bash
FILE=prodcons1.pml
spin -a $FILE
gcc pan.c
./a.out
[ -f $FILE.trail ] && spin -t $FILE 

pan:1: assertion violated (pos<5) (at depth 26)
pan: wrote prodcons1.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 48 byte, depth reached 26, errors: 1
       27 states, stored
       19 states, matched
       46 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.002	equivalent memory usage for states (stored*(State-vector + overhead))
    0.290	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0 seconds
spin: prodcons1.pml:12, Error: assertion violated
spin: text of failed assertion: assert((pos<5))
spin: trail ends after 27 steps
#processes: 2
		buffer[0] = 0
		buffer[1] = 0
		

In [100]:
%%file prodcons2.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

#define MAX 5

//int buffer[MAX]
byte pos = 0

byte notEmpty = 0
byte notFull = MAX

inline append(d) {
    assert pos < MAX
    atomic {
        //buffer[pos] = d
        pos++
    }
}

inline take(d) {
    atomic {
        pos--
        //d = buffer[pos]
    }
}

//inline produce(d) {
//    d++
//}

active proctype Producer() {
    int d
    do
    ::
      wait(notFull)
      //produce(d)
      append(d)
      signal(notEmpty)
    od
}

active proctype Consumer() {
    int d
    do
    ::
      wait(notEmpty)
      take(d)
      signal(notFull)
    od
}

Overwriting prodcons2.pml


In [101]:
%%bash
FILE=prodcons2.pml
spin -a $FILE
gcc pan.c
./a.out
[ -f $FILE.trail ] && spin -t $FILE


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 32 byte, depth reached 50, errors: 0
       55 states, stored
       49 states, matched
      104 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.003	equivalent memory usage for states (stored*(State-vector + overhead))
    0.290	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype Producer
	prodcons2.pml:40, state 12, "-end-"
	(1 of 12 states)
unreached in proctype Consumer
	prodcons2.pml:50, state 11, "-end-"
	(1 of 11 states)

pan: elapsed time 0.01 seconds


In [109]:
%%file readerswriters.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

// semaphores
byte writerAccess = 1
byte readerAccess = 1

// counter
byte numWriters = 0
byte numReaders = 0

active proctype Writer() {
    do
    ::
      wait(writerAccess)
      numWriters++
      assert numWriters <= 1 && numReaders <= 1
      numWriters--
      signal(writerAccess)
    od
}

active[4] proctype Reader() {
    do
    ::
      wait(readerAccess)
      numReaders++
      if
      :: numReaders == 1 -> wait(writerAccess)
      :: else -> skip
      fi
      signal(readerAccess)

      assert numWriters == 0

      wait(readerAccess)
      numReaders--
      if
      :: numReaders == 0 -> signal(writerAccess)
      :: else -> skip
      fi
      signal(readerAccess)
    od
}

Overwriting readerswriters.pml


In [112]:
%%bash
FILE=readerswriters.pml
rm pan*
rm *trail
spin -a $FILE && gcc pan.c && ./a.out
[ -f $FILE.trail ] && spin -t $FILE


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 52 byte, depth reached 305, errors: 0
     1013 states, stored
      976 states, matched
     1989 transitions (= stored+matched)
        0 atomic steps
hash conflicts:        23 (resolved)

Stats on memory usage (in Megabytes):
    0.077	equivalent memory usage for states (stored*(State-vector + overhead))
    0.289	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype Writer
	readerswriters.pml:21, state 11, "-end-"
	(1 of 11 states)
unreached in proctype Reader
	readerswriters.pml:44, state 29, "-end-"
	(1 of 29 states)

pan: elapsed time 0 seconds


rm: cannot remove '*trail': No such file or directory


In [132]:
%%file philosophers.pml
#define grab(s) atomic { (s) > 0 -> s = s - 1 }
#define drop(s) s = s + 1


byte forks[5] = { 1 }

active[5] proctype Philosopher() {
    byte i = _pid
    do
    :: 
      // think
      grab(forks[i])
      grab(forks[(i + 1) % 5])
      // eat
      drop(forks[i])
      drop(forks[(i + 1) % 5])
    od
}

Overwriting philosophers.pml


In [133]:
%%bash
FILE=philosophers.pml
rm pan*
rm *trail
spin -a $FILE && gcc pan.c && ./a.out
[ -f $FILE.trail ] && spin -t $FILE

pan:1: invalid end state (at depth 56)
pan: wrote philosophers.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 52 byte, depth reached 57, errors: 1
       58 states, stored
       16 states, matched
       74 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.004	equivalent memory usage for states (stored*(State-vector + overhead))
    0.291	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0 seconds
spin: trail ends after 57 steps
#processes: 5
		forks[0] = 0
		forks[1] = 0
		forks[2] = 0
		forks[3] = 0
		forks[4] = 0
 57:	proc  4 (Philosopher:1) philosophers.pml:13 (state 6)
 57:

rm: cannot remove '*trail': No such file or directory


In [130]:
%%file philosophers2.pml
#define grab(s) atomic { (s) > 0 -> s = s - 1 }
#define drop(s) s = s + 1


byte forks[5] = { 1 }

active[4] proctype Philosopher() {
    byte i = _pid
    do
    :: 
      // think
      grab(forks[i])
      grab(forks[i+1])
      // eat
      drop(forks[i])
      drop(forks[i+1])
    od
}
active proctype LeftyPhilosopher() {
    byte i = _pid
    do
    :: 
      // think
      grab(forks[0])
      grab(forks[4])
      // eat
      drop(forks[0])
      drop(forks[4])
    od
}

Overwriting philosophers2.pml


In [131]:
%%bash
FILE=philosophers2.pml
rm pan*
rm *trail
spin -a $FILE && gcc pan.c && ./a.out
[ -f $FILE.trail ] && spin -t $FILE


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 52 byte, depth reached 174, errors: 0
      243 states, stored
      568 states, matched
      811 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.019	equivalent memory usage for states (stored*(State-vector + overhead))
    0.290	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype Philosopher
	philosophers2.pml:18, state 12, "-end-"
	(1 of 12 states)
unreached in proctype LeftyPhilosopher
	philosophers2.pml:30, state 12, "-end-"
	(1 of 12 states)

pan: elapsed time 0 seconds
